In [1]:
# pip install autocorrect

In [2]:
# pip install --user -U nltk

In [1]:
import pandas as pd
import json
import numpy as np
from collections import defaultdict
from pathlib import Path
import pandas as pd
import string
import re
from nltk.corpus import stopwords
from autocorrect import Speller
from nltk.stem import WordNetLemmatizer 

In [2]:
import nltk
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('omw-1.4')

In [3]:
pd.set_option('display.max_colwidth', -1)

/var/folders/c9/z73mt59x7xndbcx483qrvg500000gn/T/ipykernel_13253/3652881321.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [ ]:
data = pd.read_csv("sample_reviews.csv")

In [4]:
display(data.head(2))
print("Size of the dataset:",data.shape)

,Unnamed: 0,review_id,user_id,business_id,text,category,n_classes
0,0,l3Wk_mvAog6XANIuGQ9C7Q,ZbqSHbgCjzVAqaa7NKWn5A,EQ-TZ2eeD_E0BHuvoaeG5Q,"Locals recommended Milktooth, and it's an amaz...","['Beer', 'Wine & Spirits', 'Cafes', 'Coffee & ...",8
1,1,vBK79c3_1Ff_oqkh5VpfGg,Ohhrhu1RkqfVciIVx_W5HQ,nRKndeZLQ3eDL10UMwS2rQ,HOLY SMOKES!\n\nactual pumpkin pie mixed in wi...,"['Ice Cream & Frozen Yogurt', 'Food', 'Local F...",4


Size of the dataset: (570442, 7)


In [5]:
type(data.text.loc[0])

str

### **Text Preprocessing**
1. Case folding
2. Removing newlines
3. Removing links
4. Removing numbers
5. Removing extra whitespaces
6. Removing emoji 
7. Tokenization + stopwords 
8. Lemmatizaion
9. Removing punctuation
10. Removing repeated characters
11. Spelling correction


#### **Case-folding**
All the text of "text" column converted into lower case 

In [6]:
data.text = data.text.str.lower()
data.head()

,Unnamed: 0,review_id,user_id,business_id,text,category,n_classes
0,0,l3Wk_mvAog6XANIuGQ9C7Q,ZbqSHbgCjzVAqaa7NKWn5A,EQ-TZ2eeD_E0BHuvoaeG5Q,"locals recommended milktooth, and it's an amaz...","['Beer', 'Wine & Spirits', 'Cafes', 'Coffee & ...",8
1,1,vBK79c3_1Ff_oqkh5VpfGg,Ohhrhu1RkqfVciIVx_W5HQ,nRKndeZLQ3eDL10UMwS2rQ,holy smokes!\n\nactual pumpkin pie mixed in wi...,"['Ice Cream & Frozen Yogurt', 'Food', 'Local F...",4
2,2,z0osLHDvXvzfm57D4DmD2Q,xVKE_HJ2pwUtTdLbL3pnCg,S2Ho8yLxhKAa26pBAm6rxA,"service was crappy, and food was mediocre. i ...","['Cajun/Creole', 'Seafood', 'Restaurants', 'Br...",4
3,3,elqRpX9T3YwL07uLNtN3Bg,-sryo4gDYxbZ1T5Bz4l5Bw,ltBBYdNzkeKdCNPDAsxwAA,i at least have to give this restaurant two st...,"['Cocktail Bars', 'Nightlife', 'Gastropubs', '...",8
4,4,pHwbdway4yeI-dSSmZA7-Q,qEEk0PuoH1dVa619t8fgpw,PY9GRfzr4nTZeINf346QOw,we checked in around 2:30 pm. check-in was qu...,"['Event Planning & Services', 'Casinos', 'Beau...",8


#### **Remove newlines**
This function will remove all the occurrences of newlines, tabs, and combinations like: `\\n`, `\\`. 
 all the occurrences of `\n,\\n,\t,\\` are replaced with a space.

In [7]:
def remove_newlines_tabs(text):
    formatted = text.replace('\\n', ' ').replace('\n', ' ').replace('\t',' ').replace('\\', ' ').replace('. com', '.com')
    return formatted

In [8]:
data.text = data.text.apply(lambda text: remove_newlines_tabs(text))

In [9]:
data.head(2)

,Unnamed: 0,review_id,user_id,business_id,text,category,n_classes
0,0,l3Wk_mvAog6XANIuGQ9C7Q,ZbqSHbgCjzVAqaa7NKWn5A,EQ-TZ2eeD_E0BHuvoaeG5Q,"locals recommended milktooth, and it's an amaz...","['Beer', 'Wine & Spirits', 'Cafes', 'Coffee & ...",8
1,1,vBK79c3_1Ff_oqkh5VpfGg,Ohhrhu1RkqfVciIVx_W5HQ,nRKndeZLQ3eDL10UMwS2rQ,holy smokes! actual pumpkin pie mixed in with...,"['Ice Cream & Frozen Yogurt', 'Food', 'Local F...",4


#### **Removing links**

Since punctuation is not removed yet, we can notice there are observations containing URLs in the text, as we can see below. 
We will remove the URLs from the text.

In [10]:
data_links = data[data['text'].str.contains(r"www\.[A-Za-z]*\.com", regex = True)] #data containing links
print("Size of the dataset with links:", data_links.shape)
display(data_links.head(2))



Size of the dataset with links: (563, 7)


,Unnamed: 0,review_id,user_id,business_id,text,category,n_classes
1025,1025,AyELQAUQjjccm1pWHwLKYw,DihTsliUxbqpoA9W3uRDBA,EtKSTHV5Qx_Q7Aur9o4kQQ,"sorry yelpers, but i thought this jose garces ...","['Bars', 'Nightlife', 'Whiskey Bars', 'Burgers...",6
2158,2158,dXj8xfsfXapb4zgH-rQK3A,k6GurG0wmHVyiZGzmxs8eQ,I6L0Zxi5Ww0zEWSAVgngeQ,i like the other cafe beignet location better ...,"['Sandwiches', 'Breakfast & Brunch', 'Cajun/Cr...",7


563 ovservations contain URLs.

In [11]:
def remove_links(text):
  https = re.sub(r'http\S+', ' ', text)
  com = re.sub(r"www\.[A-Za-z]*\.com", " ", https)
  return com

In [12]:
data['text']= data['text'].apply(remove_links)

As we can see, there are no more observations where text contains links:

In [13]:
data[data['text'].str.contains(r"www\.[A-Za-z]*\.com", regex = True)]

,Unnamed: 0,review_id,user_id,business_id,text,category,n_classes


In [14]:
data[data['text'].str.contains(r"http\S+", regex = True)]

,Unnamed: 0,review_id,user_id,business_id,text,category,n_classes


Anyway, there are still websites written just in the form ending with `.com` . As we can see below:

In [15]:
data_com = data[data['text'].str.contains(r"[A-Za-z]*\.com[^a-zA-Z]", regex = True)]  #consider after .com, any non alphanumeric symbol. 
print(data_com.shape)

(594, 7)


In [18]:
data_com.head(2)

,Unnamed: 0,review_id,user_id,business_id,text,category,n_classes
683,683,BfJsxwDZ9pruhNRfjljT0w,mkQz8aihI1V19sYH8JQy-A,8uF-bhJFgT4Tn6DTb27viA,"coming from a nyc sweets-hater - these donuts are divine. luxuriously extravagant and very creative. i had a banana pudding donut with a house brew. others on display were thai iced tea, pb&j and milk & cereal. so cute. taste is absolutely bomb.com. will make it a pit-stop next time i'm in no.","['Food', 'Donuts', 'Burgers', 'American (Traditional)', 'Coffee & Tea', 'Restaurants']",6
760,760,2mxuPqNLpEzg1LGPEMG4Fg,gqDbe1YwS4EhuCCNSNhpxg,8uF-bhJFgT4Tn6DTb27viA,bomb.com! i forget what i ordered but it was similar to a cuban sandwich on donut bread. my friend ordered a cinnamon sugar donut while she waited for her actual food and that was delicious. i ate everything because it was amazing and deserves all the hype it has received. i need to go back and try those stuffed donuts.,"['Food', 'Donuts', 'Burgers', 'American (Traditional)', 'Coffee & Tea', 'Restaurants']",6


We will remove the websites written in this form as well.

In [19]:
def remove_com(text):
 c = re.sub(r'[A-Za-z]*\.com[^a-zA-Z]', ' ', text)
 return c

In [20]:
data_com['text'] = data_com['text'].apply(remove_com)


/var/folders/c9/z73mt59x7xndbcx483qrvg500000gn/T/ipykernel_28329/1957766861.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_com['text'] = data_com['text'].apply(remove_com)


As we can see, websites ending with ".com" were removed.

In [21]:
data_com.head(2) #bomb.com removed

,Unnamed: 0,review_id,user_id,business_id,text,category,n_classes
683,683,BfJsxwDZ9pruhNRfjljT0w,mkQz8aihI1V19sYH8JQy-A,8uF-bhJFgT4Tn6DTb27viA,"coming from a nyc sweets-hater - these donuts are divine. luxuriously extravagant and very creative. i had a banana pudding donut with a house brew. others on display were thai iced tea, pb&j and milk & cereal. so cute. taste is absolutely will make it a pit-stop next time i'm in no.","['Food', 'Donuts', 'Burgers', 'American (Traditional)', 'Coffee & Tea', 'Restaurants']",6
760,760,2mxuPqNLpEzg1LGPEMG4Fg,gqDbe1YwS4EhuCCNSNhpxg,8uF-bhJFgT4Tn6DTb27viA,i forget what i ordered but it was similar to a cuban sandwich on donut bread. my friend ordered a cinnamon sugar donut while she waited for her actual food and that was delicious. i ate everything because it was amazing and deserves all the hype it has received. i need to go back and try those stuffed donuts.,"['Food', 'Donuts', 'Burgers', 'American (Traditional)', 'Coffee & Tea', 'Restaurants']",6


The function is applied to the entire dataset

In [22]:
data['text'] = data['text'].apply(remove_com)

The websites in the form `.com` are removed:

In [23]:
data[data['text'].str.contains(r"[A-Za-z]*\.com[^a-zA-Z]", regex = True)] 

,Unnamed: 0,review_id,user_id,business_id,text,category,n_classes


#### **Remove numbers**

In [24]:
data['text']= data['text'].apply(lambda x: re.sub(r'\d+', '', x))

In [25]:
data.head()

,Unnamed: 0,review_id,user_id,business_id,text,category,n_classes
0,0,l3Wk_mvAog6XANIuGQ9C7Q,ZbqSHbgCjzVAqaa7NKWn5A,EQ-TZ2eeD_E0BHuvoaeG5Q,"locals recommended milktooth, and it's an amazing jewel of indianapolis. i'm glade i had the chance to experience this.","['Beer', 'Wine & Spirits', 'Cafes', 'Coffee & Tea', 'Restaurants', 'Food', 'Breakfast & Brunch', 'American (New)']",8
1,1,vBK79c3_1Ff_oqkh5VpfGg,Ohhrhu1RkqfVciIVx_W5HQ,nRKndeZLQ3eDL10UMwS2rQ,holy smokes! actual pumpkin pie mixed in with the frozen custard......are you kidding me? why hasn't this become a huge sweep the nation treat. the best part was the chunks of pie crust. like finding a diamond inside another diamond. super yummy.,"['Ice Cream & Frozen Yogurt', 'Food', 'Local Flavor', 'Desserts']",4
2,2,z0osLHDvXvzfm57D4DmD2Q,xVKE_HJ2pwUtTdLbL3pnCg,S2Ho8yLxhKAa26pBAm6rxA,"service was crappy, and food was mediocre. i wish i would have picked some other place for my last dinner in town.","['Cajun/Creole', 'Seafood', 'Restaurants', 'Breakfast & Brunch']",4
3,3,elqRpX9T3YwL07uLNtN3Bg,-sryo4gDYxbZ1T5Bz4l5Bw,ltBBYdNzkeKdCNPDAsxwAA,"i at least have to give this restaurant two stars due to the decent food. but while on a dinner meeting and after spending $ on entrées and appetizers, i had to flag down the assistant cook, (the guy in bright red baggy pants watching tv), to see if i could get more avocado for my tuna entrée. he made it very apparent and direct that he could not help me with that situation and i would have to flag down my waitress so she could bill me for the four dollar avocado.","['Cocktail Bars', 'Nightlife', 'Gastropubs', 'Sports Bars', 'Restaurants', 'American (New)', 'Lounges', 'Bars']",8
4,4,pHwbdway4yeI-dSSmZA7-Q,qEEk0PuoH1dVa619t8fgpw,PY9GRfzr4nTZeINf346QOw,"we checked in around : pm. check-in was quick and easy with complimentary valet parking (always appreciated). the peppermill is huge and reminded us of the caesars palace in las vegas. they have around eight restaurants on site offering almost any type of food you could want. the casino area is huge, rivaling almost anything you could find in las vegas. one thing we noticed is that many of the slot machines were noted as being penny machines, but when you actually started to play them you realized they had minimum bets of anywhere from cents to $. talk about false advertising! we were in the tuscany tower on the th floor. our room had a nice view of the huge pool area. our room was large and featured free wifi (likewise always appreciated), a huge bathroom area with large shower (with two shower heads), tv, a large jetted bathtub and a phone next to the commode. there were dual sinks and lots of counter space. plenty of towels were supplied. the roomy main area had a king bed with night stands, closet, tv, mini fridge, dresser with two drawers, a safe, table for computer or writing, complimentary bottles of water, and three nicely upholstered chairs and table. the thermostat did a good job controlling the temperature and keeping it comfortable. staff were all courteous and helpful. a very nice place and we enjoyed our stay.","['Event Planning & Services', 'Casinos', 'Beauty & Spas', 'Venues & Event Spaces', 'Arts & Entertainment', 'Day Spas', 'Hotels', 'Hotels & Travel']",8


#### **Remove whitespaces**
To remove extrawhitespaces

In [27]:
def remove_whitespaces(text):
    return " ".join(text.split())

In [28]:
data['text'] = data['text'].apply(lambda x: remove_whitespaces(x))

#### Removing emojis

In [29]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    try:
         emoji_pattern.sub(r' ', string)
         return emoji_pattern.sub(r' ', string)
    except:
        print(string)
        return string
    

The function is applied to the dataset we are working with:

In [30]:
data['text'] = data['text'].apply(lambda x: remove_emoji(x))

#### Tokenization + stopwords removal

In the following steps, we will use the SpaCy library to tokenize the text, aplly stopwords and puctuation removal, storing the result in a new column.

In [31]:
import spacy
#load the spaCy English language model
nlp=spacy.load('en_core_web_sm')

In [32]:
#doc = nlp(str(data['text'])) #convert our text data into doc object with tokens
#print(type(doc))

<class 'spacy.tokens.doc.Doc'>


To start processing the input text, we construct a Doc object.
A Doc object is a sequence of Token objects representing a lexical token. Each Token object has information about a particular piece — typically one word — of text.

SpaCy stores a list of stop words for the English language:

In [35]:
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS
print("Number of stopwords:", len(spacy_stopwords))

Number of stopwords: 326


In [36]:
for stop_word in list(spacy_stopwords)[:10]:
 print(stop_word)

during
former
meanwhile
whither
anyone
while
besides
one
alone
only


In [43]:
def process_text(text):
    # Process the text using SpaCy
    doc = nlp(str(text))
    
    # Remove stop words
    tokens = [token.text for token in doc if not token.is_stop and not token.is_punct and token.text != "€" and token.text != "$"]
    
    # Join the tokens into a single string
    processed_text = ' '.join(tokens)
    
    return processed_text


In [22]:
import time

In [23]:
from pandarallel import pandarallel #to parallelize operations on dataframe for efficiency improving

In [24]:
pandarallel.initialize( #use 4-core to speed up the process
    nb_workers=4,
    progress_bar=True
)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [57]:
%%time
data['token'] = data['text'].parallel_apply(process_text)


CPU times: user 45.4 s, sys: 9.63 s, total: 55 s
Wall time: 1h 22min 9s


In [59]:
#data.to_csv("data_token.csv", index=False) 

In [60]:
data.head()

,Unnamed: 0,review_id,user_id,business_id,text,category,n_classes,token
0,0,l3Wk_mvAog6XANIuGQ9C7Q,ZbqSHbgCjzVAqaa7NKWn5A,EQ-TZ2eeD_E0BHuvoaeG5Q,"locals recommended milktooth, and it's an amazing jewel of indianapolis. i'm glade i had the chance to experience this.","['Beer', 'Wine & Spirits', 'Cafes', 'Coffee & Tea', 'Restaurants', 'Food', 'Breakfast & Brunch', 'American (New)']",8,locals recommended milktooth amazing jewel indianapolis glade chance experience
1,1,vBK79c3_1Ff_oqkh5VpfGg,Ohhrhu1RkqfVciIVx_W5HQ,nRKndeZLQ3eDL10UMwS2rQ,holy smokes! actual pumpkin pie mixed in with the frozen custard......are you kidding me? why hasn't this become a huge sweep the nation treat. the best part was the chunks of pie crust. like finding a diamond inside another diamond. super yummy.,"['Ice Cream & Frozen Yogurt', 'Food', 'Local Flavor', 'Desserts']",4,holy smokes actual pumpkin pie mixed frozen custard kidding huge sweep nation treat best chunks pie crust like finding diamond inside diamond super yummy
2,2,z0osLHDvXvzfm57D4DmD2Q,xVKE_HJ2pwUtTdLbL3pnCg,S2Ho8yLxhKAa26pBAm6rxA,"service was crappy, and food was mediocre. i wish i would have picked some other place for my last dinner in town.","['Cajun/Creole', 'Seafood', 'Restaurants', 'Breakfast & Brunch']",4,service crappy food mediocre wish picked place dinner town
3,3,elqRpX9T3YwL07uLNtN3Bg,-sryo4gDYxbZ1T5Bz4l5Bw,ltBBYdNzkeKdCNPDAsxwAA,"i at least have to give this restaurant two stars due to the decent food. but while on a dinner meeting and after spending $ on entrées and appetizers, i had to flag down the assistant cook, (the guy in bright red baggy pants watching tv), to see if i could get more avocado for my tuna entrée. he made it very apparent and direct that he could not help me with that situation and i would have to flag down my waitress so she could bill me for the four dollar avocado.","['Cocktail Bars', 'Nightlife', 'Gastropubs', 'Sports Bars', 'Restaurants', 'American (New)', 'Lounges', 'Bars']",8,restaurant stars decent food dinner meeting spending entrées appetizers flag assistant cook guy bright red baggy pants watching tv avocado tuna entrée apparent direct help situation flag waitress bill dollar avocado
4,4,pHwbdway4yeI-dSSmZA7-Q,qEEk0PuoH1dVa619t8fgpw,PY9GRfzr4nTZeINf346QOw,"we checked in around : pm. check-in was quick and easy with complimentary valet parking (always appreciated). the peppermill is huge and reminded us of the caesars palace in las vegas. they have around eight restaurants on site offering almost any type of food you could want. the casino area is huge, rivaling almost anything you could find in las vegas. one thing we noticed is that many of the slot machines were noted as being penny machines, but when you actually started to play them you realized they had minimum bets of anywhere from cents to $. talk about false advertising! we were in the tuscany tower on the th floor. our room had a nice view of the huge pool area. our room was large and featured free wifi (likewise always appreciated), a huge bathroom area with large shower (with two shower heads), tv, a large jetted bathtub and a phone next to the commode. there were dual sinks and lots of counter space. plenty of towels were supplied. the roomy main area had a king bed with night stands, closet, tv, mini fridge, dresser with two drawers, a safe, table for computer or writing, complimentary bottles of water, and three nicely upholstered chairs and table. the thermostat did a good job controlling the temperature and keeping it comfortable. staff were all courteous and helpful. a very nice place and we enjoyed our stay.","['Event Planning & Services', 'Casinos', 'Beauty & Spas', 'Venues & Event Spaces', 'Arts & Entertainment', 'Day Spas', 'Hotels', 'Hotels & Travel']",8,checked pm check quick easy complimentary valet parking appreciated peppermill huge reminded caesars palace las vegas restaurants site offering type food want casino area huge rivaling find las vegas thing noticed slot mach

#### **SpaCy Lemmatizer**

In [62]:
def lemma_spacy(text): # input: tokens, tipo stringa
    doc = nlp(text)
    # lemmatization
    lemmas = [token.lemma_ for token in doc]
    # join lemmas into a single string
    lemma_text = ' '.join(lemmas)
    
    return lemma_text

In [63]:
%%time
data['lemma'] = data['token'].parallel_apply(lemma_spacy) 


CPU times: user 24 s, sys: 5.26 s, total: 29.2 s
Wall time: 36min 6s


In [65]:
#data.to_csv("data_preprocessed.csv",index=False)

WordNet Lemmatizer

In [4]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

In [6]:
def lemmatizer(text):
  lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
  return lemm_text


#### Remaing preprocessing 

In [5]:
data = pd.read_csv("data_preprocessed.csv")

In [56]:
data.shape

(570442, 9)

In [65]:
data.drop(columns=['Unnamed: 0'])

,review_id,user_id,business_id,text,category,n_classes,token,lemma
0,l3Wk_mvAog6XANIuGQ9C7Q,ZbqSHbgCjzVAqaa7NKWn5A,EQ-TZ2eeD_E0BHuvoaeG5Q,"locals recommended milktooth, and it's an amazing jewel of indianapolis. i'm glade i had the chance to experience this.","['Beer', 'Wine & Spirits', 'Cafes', 'Coffee & Tea', 'Restaurants', 'Food', 'Breakfast & Brunch', 'American (New)']",8,locals recommended milktooth amazing jewel indianapolis glade chance experience,local recommend milktooth amazing jewel indianapolis grade chance experience
1,vBK79c3_1Ff_oqkh5VpfGg,Ohhrhu1RkqfVciIVx_W5HQ,nRKndeZLQ3eDL10UMwS2rQ,holy smokes! actual pumpkin pie mixed in with the frozen custard......are you kidding me? why hasn't this become a huge sweep the nation treat. the best part was the chunks of pie crust. like finding a diamond inside another diamond. super yummy.,"['Ice Cream & Frozen Yogurt', 'Food', 'Local Flavor', 'Desserts']",4,holy smokes actual pumpkin pie mixed frozen custard kidding huge sweep nation treat best chunks pie crust like finding diamond inside diamond super yummy,holy smoke actual pumpkin pie mix frozen custard kid huge sweep nation treat well chunk pie crust like find diamond inside diamond super dummy
2,z0osLHDvXvzfm57D4DmD2Q,xVKE_HJ2pwUtTdLbL3pnCg,S2Ho8yLxhKAa26pBAm6rxA,"service was crappy, and food was mediocre. i wish i would have picked some other place for my last dinner in town.","['Cajun/Creole', 'Seafood', 'Restaurants', 'Breakfast & Brunch']",4,service crappy food mediocre wish picked place dinner town,service crappy food mediocre wish pick place dinner town
3,elqRpX9T3YwL07uLNtN3Bg,-sryo4gDYxbZ1T5Bz4l5Bw,ltBBYdNzkeKdCNPDAsxwAA,"i at least have to give this restaurant two stars due to the decent food. but while on a dinner meeting and after spending $ on entrées and appetizers, i had to flag down the assistant cook, (the guy in bright red baggy pants watching tv), to see if i could get more avocado for my tuna entrée. he made it very apparent and direct that he could not help me with that situation and i would have to flag down my waitress so she could bill me for the four dollar avocado.","['Cocktail Bars', 'Nightlife', 'Gastropubs', 'Sports Bars', 'Restaurants', 'American (New)', 'Lounges', 'Bars']",8,restaurant stars decent food dinner meeting spending entrées appetizers flag assistant cook guy bright red baggy pants watching tv avocado tuna entrée apparent direct help situation flag waitress bill dollar avocado,restaurant star decent food dinner meeting spending entryées appetite flag assistant cook guy bright red buggy part watch tv avocado tuna entryée apparent direct help situation flag waitress bill dollar avocado
4,pHwbdway4yeI-dSSmZA7-Q,qEEk0PuoH1dVa619t8fgpw,PY9GRfzr4nTZeINf346QOw,"we checked in around : pm. check-in was quick and easy with complimentary valet parking (always appreciated). the peppermill is huge and reminded us of the caesars palace in las vegas. they have around eight restaurants on site offering almost any type of food you could want. the casino area is huge, rivaling almost anything you could find in las vegas. one thing we noticed is that many of the slot machines were noted as being penny machines, but when you actually started to play them you realized they had minimum bets of anywhere from cents to $. talk about false advertising! we were in the tuscany tower on the th floor. our room had a nice view of the huge pool area. our room was large and featured free wifi (likewise always appreciated), a huge bathroom area with large shower (with two shower heads), tv, a large jetted bathtub and a phone next to the commode. there were dual sinks and lots of counter space. plenty of towels were supplied. the roomy main area had a king bed with night stands, closet, tv, mini fridge, dresser with two drawers, a safe, table for computer or writing, complimentary bottles of water, and three nicely upholstered chairs and table. the thermostat did a good job controlling the temperature and keeping 

In [57]:
data = data.dropna()

In [58]:
data.shape

(570438, 9)

#### Punctuation removal

In [59]:
punct = list(string.punctuation)
punct.append(['€','¯'])

In [60]:
print("Punctuation to remove:", string.punctuation, "€ ¯ ")

Punctuation to remove: !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ € ¯ 


In [61]:
def remove_punctuation(text):
    punctuationfree="".join([w for w in text if w not in punct])
    return punctuationfree

In [62]:
data['lemma']= data['lemma'].apply(remove_punctuation)

#### Removing repeated characters

In [63]:
def removing_character_repetition(text):
    pattern = re.compile(r"([A-Za-z])\1{1,}", re.DOTALL) #matching for all case alphabets
    formatted = pattern.sub(r"\1\1", text) #limits to two characters
    return formatted

In [18]:
data['lemma'] = data['lemma'].apply(removing_character_repetition)

#### Spelling correction

In [20]:
def spelling_correction(text):
 spell = Speller(lang='en')
 corrected_text = spell(text)
 return corrected_text

In [64]:
%%time
data['lemma'] = data['lemma'].parallel_apply(spelling_correction)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pandarallel/data_types/series.py:16: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  yield data[chunk_]


CPU times: user 1min 59s, sys: 25.2 s, total: 2min 24s
Wall time: 6h 24min 9s


In [66]:
data.head()

,Unnamed: 0,review_id,user_id,business_id,text,category,n_classes,token,lemma
0,0,l3Wk_mvAog6XANIuGQ9C7Q,ZbqSHbgCjzVAqaa7NKWn5A,EQ-TZ2eeD_E0BHuvoaeG5Q,"locals recommended milktooth, and it's an amazing jewel of indianapolis. i'm glade i had the chance to experience this.","['Beer', 'Wine & Spirits', 'Cafes', 'Coffee & Tea', 'Restaurants', 'Food', 'Breakfast & Brunch', 'American (New)']",8,locals recommended milktooth amazing jewel indianapolis glade chance experience,local recommend milktooth amazing jewel indianapolis grade chance experience
1,1,vBK79c3_1Ff_oqkh5VpfGg,Ohhrhu1RkqfVciIVx_W5HQ,nRKndeZLQ3eDL10UMwS2rQ,holy smokes! actual pumpkin pie mixed in with the frozen custard......are you kidding me? why hasn't this become a huge sweep the nation treat. the best part was the chunks of pie crust. like finding a diamond inside another diamond. super yummy.,"['Ice Cream & Frozen Yogurt', 'Food', 'Local Flavor', 'Desserts']",4,holy smokes actual pumpkin pie mixed frozen custard kidding huge sweep nation treat best chunks pie crust like finding diamond inside diamond super yummy,holy smoke actual pumpkin pie mix frozen custard kid huge sweep nation treat well chunk pie crust like find diamond inside diamond super dummy
2,2,z0osLHDvXvzfm57D4DmD2Q,xVKE_HJ2pwUtTdLbL3pnCg,S2Ho8yLxhKAa26pBAm6rxA,"service was crappy, and food was mediocre. i wish i would have picked some other place for my last dinner in town.","['Cajun/Creole', 'Seafood', 'Restaurants', 'Breakfast & Brunch']",4,service crappy food mediocre wish picked place dinner town,service crappy food mediocre wish pick place dinner town
3,3,elqRpX9T3YwL07uLNtN3Bg,-sryo4gDYxbZ1T5Bz4l5Bw,ltBBYdNzkeKdCNPDAsxwAA,"i at least have to give this restaurant two stars due to the decent food. but while on a dinner meeting and after spending $ on entrées and appetizers, i had to flag down the assistant cook, (the guy in bright red baggy pants watching tv), to see if i could get more avocado for my tuna entrée. he made it very apparent and direct that he could not help me with that situation and i would have to flag down my waitress so she could bill me for the four dollar avocado.","['Cocktail Bars', 'Nightlife', 'Gastropubs', 'Sports Bars', 'Restaurants', 'American (New)', 'Lounges', 'Bars']",8,restaurant stars decent food dinner meeting spending entrées appetizers flag assistant cook guy bright red baggy pants watching tv avocado tuna entrée apparent direct help situation flag waitress bill dollar avocado,restaurant star decent food dinner meeting spending entryées appetite flag assistant cook guy bright red buggy part watch tv avocado tuna entryée apparent direct help situation flag waitress bill dollar avocado
4,4,pHwbdway4yeI-dSSmZA7-Q,qEEk0PuoH1dVa619t8fgpw,PY9GRfzr4nTZeINf346QOw,"we checked in around : pm. check-in was quick and easy with complimentary valet parking (always appreciated). the peppermill is huge and reminded us of the caesars palace in las vegas. they have around eight restaurants on site offering almost any type of food you could want. the casino area is huge, rivaling almost anything you could find in las vegas. one thing we noticed is that many of the slot machines were noted as being penny machines, but when you actually started to play them you realized they had minimum bets of anywhere from cents to $. talk about false advertising! we were in the tuscany tower on the th floor. our room had a nice view of the huge pool area. our room was large and featured free wifi (likewise always appreciated), a huge bathroom area with large shower (with two shower heads), tv, a large jetted bathtub and a phone next to the commode. there were dual sinks and lots of counter space. plenty of towels were supplied. the roomy main area had a king bed with night stands, closet, tv, mini fridge, dresser with two drawers, a safe, table for computer or writing, complimentary bottles of water, and three nicely upholstered chairs and table. the thermostat did a good job controlling the tem

In [67]:
data.to_csv("data_preprocessed2.csv", index=False)

In [8]:
# data = pd.read_csv("data_preprocessed2.csv")

In [41]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

In [42]:
def lemmatizer(text):
  lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text.split()]
  return lemm_text

In [43]:
data['lemma_text'] = data['token'].apply(lemmatizer) #wordnet lemmatizer

In [44]:
data.head()

,Unnamed: 0,review_id,user_id,business_id,text,category,n_classes,token,lemma,lemma_text
0,0,l3Wk_mvAog6XANIuGQ9C7Q,ZbqSHbgCjzVAqaa7NKWn5A,EQ-TZ2eeD_E0BHuvoaeG5Q,"locals recommended milktooth, and it's an amazing jewel of indianapolis. i'm glade i had the chance to experience this.","['Beer', 'Wine & Spirits', 'Cafes', 'Coffee & Tea', 'Restaurants', 'Food', 'Breakfast & Brunch', 'American (New)']",8,locals recommended milktooth amazing jewel indianapolis glade chance experience,local recommend milktooth amazing jewel indianapolis grade chance experience,"[local, recommended, milktooth, amazing, jewel, indianapolis, glade, chance, experience]"
1,1,vBK79c3_1Ff_oqkh5VpfGg,Ohhrhu1RkqfVciIVx_W5HQ,nRKndeZLQ3eDL10UMwS2rQ,holy smokes! actual pumpkin pie mixed in with the frozen custard......are you kidding me? why hasn't this become a huge sweep the nation treat. the best part was the chunks of pie crust. like finding a diamond inside another diamond. super yummy.,"['Ice Cream & Frozen Yogurt', 'Food', 'Local Flavor', 'Desserts']",4,holy smokes actual pumpkin pie mixed frozen custard kidding huge sweep nation treat best chunks pie crust like finding diamond inside diamond super yummy,holy smoke actual pumpkin pie mix frozen custard kid huge sweep nation treat well chunk pie crust like find diamond inside diamond super dummy,"[holy, smoke, actual, pumpkin, pie, mixed, frozen, custard, kidding, huge, sweep, nation, treat, best, chunk, pie, crust, like, finding, diamond, inside, diamond, super, yummy]"
2,2,z0osLHDvXvzfm57D4DmD2Q,xVKE_HJ2pwUtTdLbL3pnCg,S2Ho8yLxhKAa26pBAm6rxA,"service was crappy, and food was mediocre. i wish i would have picked some other place for my last dinner in town.","['Cajun/Creole', 'Seafood', 'Restaurants', 'Breakfast & Brunch']",4,service crappy food mediocre wish picked place dinner town,service crappy food mediocre wish pick place dinner town,"[service, crappy, food, mediocre, wish, picked, place, dinner, town]"
3,3,elqRpX9T3YwL07uLNtN3Bg,-sryo4gDYxbZ1T5Bz4l5Bw,ltBBYdNzkeKdCNPDAsxwAA,"i at least have to give this restaurant two stars due to the decent food. but while on a dinner meeting and after spending $ on entrées and appetizers, i had to flag down the assistant cook, (the guy in bright red baggy pants watching tv), to see if i could get more avocado for my tuna entrée. he made it very apparent and direct that he could not help me with that situation and i would have to flag down my waitress so she could bill me for the four dollar avocado.","['Cocktail Bars', 'Nightlife', 'Gastropubs', 'Sports Bars', 'Restaurants', 'American (New)', 'Lounges', 'Bars']",8,restaurant stars decent food dinner meeting spending entrées appetizers flag assistant cook guy bright red baggy pants watching tv avocado tuna entrée apparent direct help situation flag waitress bill dollar avocado,restaurant star decent food dinner meeting spending entryées appetite flag assistant cook guy bright red buggy part watch tv avocado tuna entryée apparent direct help situation flag waitress bill dollar avocado,"[restaurant, star, decent, food, dinner, meeting, spending, entrées, appetizer, flag, assistant, cook, guy, bright, red, baggy, pant, watching, tv, avocado, tuna, entrée, apparent, direct, help, situation, flag, waitress, bill, dollar, avocado]"
4,4,pHwbdway4yeI-dSSmZA7-Q,qEEk0PuoH1dVa619t8fgpw,PY9GRfzr4nTZeINf346QOw,"we checked in around : pm. check-in was quick and easy with complimentary valet parking (always appreciated). the peppermill is huge and reminded us of the caesars palace in las vegas. they have around eight restaurants on site offering almost any type of food you could want. the casino area is huge, rivaling almost anything you could find in las vegas. one thing we noticed is that many of the slot machines were noted as being penny machines, but when you actually started to play them you realized they had minimum bets of anywhere from cents to $. talk about false advertising! we were in the tuscany tower on the th floor.

In [46]:
data.to_csv("data_preprocessed3.csv")